In [ ]:
import numpy as np
import pickle
import sys

In [ ]:
BOARD_ROWS = 7
BOARD_COLS = 7

In [ ]:
 # 참고 https://www.secmem.org/blog/2020/02/08/snake-dqn/

In [ ]:
class State:
  def __init__(self, p1, p2):
    self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
    self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
    self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = -1
    self.p1 = p1
    self.p2 = p2
    self.isEnd = False
    self.boardHash = None
    # init p1 plays first
    self.playerSymbol = 1

  def cantmove(self): # 더 이상 움직일 수 없을 때 남은 곳을 상대 말로 채운다.
    for i in range(BOARD_ROWS):
      for j in range(BOARD_COLS):
        if self.board[i, j] == 0:
          self.board[i, j] = -self.playerSymbol
    return None

  def winner(self): # 맵이 다 찼다면 점수를 반환한다.
    if sum(map(sum, map(abs, self.board))) == BOARD_ROWS*BOARD_COLS:
      self.isEnd = True
      return sum(map(sum, self.board))
    self.isEnd = False
    return None

  def availableActions(self): # 가능한 행동들을 반환한다.
    Actions = []
    for i in range(BOARD_ROWS):
        for j in range(BOARD_COLS):
            if self.board[i, j] == self.playerSymbol:
                for ii in range(-2,3):
                    for jj in range(-2,3):
                        if ii == 0 and jj == 0:
                          continue
                        if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                          continue
                        if self.board[i + ii, j + jj] == 0:
                          act = i
                          act = act*BOARD_COLS + j
                          act = act*BOARD_COLS + i + ii
                          act = act*BOARD_COLS + j + jj
                          Actions.append(act)
    return Actions


  def isAvailabeAction(self, Action): # 가능한 행동인지?
    position = []
        for i in range(4):
          position[3-i] = Action % BOARD_COLS
          Action /= BOARD_COLS
    return self.board[position[0],position[1]]==self.playerSymbol and self.board[position[2],position[3]]==0


  def updateState(self, Action): # 현재 상태에서 특정 행동을 한 다음 상태로 업데이트 한다.
      position = []
      for i in range(4):
        position[3-i] = Action % BOARD_COLS
        Action /= BOARD_COLS
      ii = position[2] - position[0]
      jj = position[3] - position[1]
      if max(abs(ii), abs(jj)) == 2:
          self.board[position[0:2]] = 0
      
      dx1 = [-1, -1, -1, 0, 0, 1, 1, 1]
      dy1 = [-1, 0, 1, -1, 1, -1, 0, 1]
      i, j = position[2:4]
      self.board[i, j] = self.playerSymbol
      for ii, jj in zip(dx1, dy1):
          if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
              continue
          if self.board[i + ii, j + jj] == -self.playerSymbol:
              self.board[i + ii, j + jj] = self.playerSymbol
          
      # switch to another player
      self.playerSymbol = -1 if self.playerSymbol == 1 else 1

  def giveReward(self): # 보상을 준다.
      result = self.winner()
      # backpropagate reward
      self.p1.feedReward(result)
      self.p2.feedReward(-result)

  def reset(self): # 리셋.
      self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
      self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
      self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = -1
      self.boardHash = None
      self.isEnd = False
      self.playerSymbol = 1
  def test(self):
    while not self.isEnd:
      avac = availableActions()
      if not avac:
        self.cantmove()
      else:
        state = self.board
        state = np.float32(state)
        q_vals = self.p1.model.predict(state)
        opt_action = 0
        for action in self.availableActions:
          if opt_action == 0:
            opt_action = action
          elif q_vals[opt_action]<q_vals[action]:
            opt_action = action
        self.updateState(opt_action)
      self.showBoard()
      win = self.winner()
      if win is not None:
        if win > 0:
          print(self.p1.name, "wins!")
        else:
          print(self.p2.name, "wins!")
        self.reset()
        break
      
      else:
        avac = availableActions()
        if not avac:
          self.cantmove()
        else:
          state = self.board
          state = np.float32(state)
          q_vals = self.p2.model.predict(state)
          opt_action == 0
          for action in self.availableActions:
            if opt_action == 0:
              opt_action = action
            elif q_vals[opt_action]<q_vals[action]:
              opt_action = action
          self.updateState(opt_action)
        self.showBoard()
        win = self.winner()
        if win is not None:
          if win > 0:
            print(self.p2.name, "wins!")
          else:
            print(self.p1.name, "wins!")
          print()
          self.reset()
          break
        
  def showBoard(self):
    # p1: o  p2: x
        for i in range(0, BOARD_ROWS):
            print('------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'o'
                if self.board[i, j] == -1:
                    token = 'x'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('------------------------------')  



    

In [ ]:
class Player:
  def __init__(self, name, alpha=0.2, gamma=0.95, epsilon=0.1):
    self.name = name
    self.alpha = alpha
    self.gamma = gamma
    self.epsilon = epsilon
    self.batch_size = 1000
    self.min_replay_memory_size = 1000 # 얼마가 적당할지 잘 모르겠음.
    self.target_update_freq = 100

    self.model = self.build_model()
    self.target_model = self.build_model()
    self.target_model.set_weights(self.model.get_weights())
    self.model.summary()

    self.replay_memory = deque(maxlen=replay_memory_size)
    self.target_update_counter = 0
    


  def build_model(self): # DQN 모델을 생성한다.
    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(7, 7), activation='relu'))
    model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(5, 5), activation='relu'))
    model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(3, 3), activation='relu'))
    model.add(Dense(64 * BOARD_COLS * BOARD_COLS, activation='relu'))
    model.add(Dense(BOARD_COLS**4, activation='relu'))
    print(model.summary())
    model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
    return model

  def update_replay_memory(self, current_state, action, reward, next_state, done): # 리플레이 메모리에 상황을 저장한다.
    self.replay_memory.append((current_state, action, reward, next_state, done))

  def get_q_values(self, x): # 현재 상태에서 할 행동들의 q_value를 반환, x는 board에 대응됨.
    return self.model.predict(x)

  def getAction(self, state, epsilon):
    if np.random.rand() <= epsilon:
       # 무작위 행동 반환
      return random.randrange(self.action_size)
    else:
       # 모델로부터 행동 산출
      state = np.float32(state)
      q_values = self.model.predict(state)
      return np.argmax(q_values[0])

  def train(self):
    if len(self.replay_memory)<self.min_replay_memory_size: # 충분히 모이지 않으면 학습하지 않는다.
      return
    
    samples = random.sample(self.replay_memory, self.batch_size)
    current_input = np.stack([sample[0] for sample in samples]) # current_state들의 array
    current_q_values = self.model.predict(current_input)
    next_input = np.stack([sample[3] for sample in samples])
    next_q_values = self.target_model.predict(next_input)

    for i, (current_state, action, reward, _, done) in enumerate(samples):
      if done:
        next_q_value = reward
      else:
        next_q_value = reward + self.gamma * np.max(next_q_values[i])
      current_q_values[i, action] = next_q_value

    hist = self.model.fit(current_input, current_q_values, batch_size=self.batch_size, verbose=0, shuffle=False)
    loss = hist.history['loss'][0]
    return loss

  def increase_target_update_counter(self): # target_model에 model을 업데이트한다. 그걸 세는 함수.
    self.target_update_counter += 1
    if self.target_update_counter >= self.target_update_freq:
      self.target_model.set_weights(self.model.get_weights())
      self.target_update_counter = 0

  def save(self, model_filepath, target_model_filepath):
    self.model.save(model_filepath)
    self.target_model.save(target_model_filepath)

  def load(self, model_filepath, target_model_filepath):
    self.model = keras.models.load_model(model_filepath)
    self.target_model = keras.models.load_model(target_model_filepath)


In [ ]:
if __name__ ==" __main__":
  agent = Player()
  agent.build_model()
  